In [ ]:
import requests
import pandas as pd
import json
import plotly.express as px
from plotly.subplots import make_subplots


# Obtenemos las claves de acceso a la API
url = "https://api.invertironline.com/token"

payload = 'username={tu_usuario}&password={tu_contraseña}*&grant_type=password'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': '1ea603=90N+IRNhzS+ZAlji2JeWwwFHYqnB5ZldfmpPr25mmXPfJx3JyDKWDLm2ANNR0cnLgvJJGe0qOY6g/L/4ppP/uvsPRie6OCjdqveFKwQxSr1ezBsEsymWYUkPcg9hDa8WUnpyEbGvdOsRU3DCvgzlkClGmeRHinBuMlRNLwRye7fv0V3+'
}

response = requests.request("POST", url, headers=headers, data=payload)
# Convertimos la respuesta a JSON
response_json = response.json()

# KEYS
bearer_token = response_json["access_token"]

In [33]:
simbolo = "come"
fecha_desde = "2024-9-29"
fecha_hasta = "2024-10-29"

def obtener_cotizacion_historica(simbolo, fecha_desde, fecha_hasta):
    
    url = f"https://api.invertironline.com/api/bCBA/Titulos/{simbolo}/Cotizacion/seriehistorica/{fecha_desde}/{fecha_hasta}/ajustada"


    payload = {}
    headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Cookie': 'i18n.langtag=es-AR; isMobile=0; 1ea603=RUJg/8xgdIOKbvv8YWewLoIwzSsqb1kcO/xLw/iEZuaPNOqbHbxikNur0VZVmNyDQBzjPVlGNLarcJk8SwAsOYxXLRN8/RKqWE15oJN16ryHneiEYrH1s9tQ6brALKC1DHjRcjMf1snj9OTuhDVgdnLBNpsLnZrQyRWsTND1JwSg6C7b'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.json()

data_historica_come = obtener_cotizacion_historica(simbolo, fecha_desde, fecha_hasta)

#print(json.dumps(data_historica_come, indent=2))

fecha = []
ultimo_precio = []
volumen_nominal = []

for index, data in enumerate(data_historica_come):
    fecha_i = data["fechaHora"]
    fecha.append(fecha_i)
    
    ultimo_precio_i = data["ultimoPrecio"]
    ultimo_precio.append(ultimo_precio_i)
    
    volumen_nominal_i = data["volumenNominal"]
    volumen_nominal.append(volumen_nominal_i)

df_historico = pd.DataFrame({
    "Fecha" : fecha,
    "Ultimo precio" : ultimo_precio,
    "Volumen nominal" : volumen_nominal
})


tendencia_fig = px.line(df_historico, 
                        x="Fecha", y="Ultimo precio")

volumen_fig = px.line(df_historico, 
                        x="Fecha", y="Volumen nominal")


fig_informe = make_subplots(rows=2, cols=1, 
                            subplot_titles=("Evolución del precio", "Evolución del volumen nominal"),
                            vertical_spacing=0.15)

for var in tendencia_fig.data:
    fig_informe.add_trace(var, row=1, col=1)
for vol in volumen_fig.data:
    fig_informe.add_trace(vol, row=2, col=1)


fig_informe.update_layout(
    title={
        'text': f"Informe de {simbolo} - Precio y volumen históricos<br><span style='font-size:13px'>desde {fecha_desde} hasta {fecha_hasta}</span>",
        'x': 0.5,  # Centra el título
        'xanchor': 'center'
    },
    showlegend=True,
    height=700,
    margin={'t': 160},
    template="plotly_dark")

# Configuración de los ejes x para mejorar la legibilidad
fig_informe.update_xaxes(
    row=1, col=1,    # Aplicar en el primer gráfico
    showgrid=False)

fig_informe.update_xaxes(
    row=2, col=1,    # Aplicar en el segundo gráfico
    showgrid=False)

# Configuración de los ejes y
fig_informe.update_yaxes(title_text="Precio ($)", row=1, col=1)

fig_informe.update_yaxes(title_text="Volumen nominal", row=2, col=1)

fig_informe.show()
